アンサンブル用シート

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib

###訓練データの読み込み
house_age = pd.read_csv('house_age.csv')
area_size = pd.read_csv('area_size.csv')
room_arrange = pd.read_csv('room_arrange.csv')
contract_span = pd.read_csv('contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('reg_rent.csv')
floor_scores = pd.read_csv('floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('stations.csv')
minits = pd.read_csv('minits.csv')
addresses = pd.read_csv('addresses.csv')
room_arrange = pd.read_csv('room_arrange.csv')
buildings = pd.read_csv('buildings.csv')

rent = pd.read_csv('rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('test_house_age.csv')
test_area_size = pd.read_csv('test_area_size.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_contract_span = pd.read_csv('test_contract_span.csv')
test_contract_span2 = pd.read_csv('test_contract_span2.csv')
test_reg_rent = pd.read_csv('test_reg_rent.csv')
test_floor_scores = pd.read_csv('test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('test_capital_floor_scores2.csv')
test_stations = pd.read_csv('test_stations.csv')
test_minits = pd.read_csv('test_minits.csv')
test_addresses = pd.read_csv('test_addresses.csv')
test_room_arrange = pd.read_csv('test_room_arrange.csv')
test_buildings = pd.read_csv('test_buildings.csv')

In [71]:
ensemble_result = []
ensemble_result = pd.DataFrame(ensemble_result)

lightGBM_result = pd.read_csv('LGBM_cs2_nLDK_result.csv')
DART_result = pd.read_csv('DART_cs_nLDKS_bld_result.csv')
GOSS_result = pd.read_csv('GOSS_cs_nLDKS_bld_result.csv')
RF_result = pd.read_csv('RF_cs2_result.csv')
MLR_result = pd.read_csv('MLR_result.csv')

ensemble_result = (lightGBM_result['0'] + DART_result['0']+ GOSS_result['0'] + RF_result['0'] + MLR_result['0'])/5
#ensemble_result = (lightGBM_result1['0'] + DART_result['0'])/2
ensemble_result.reset_index()

,index,0
0,0,137317.640924
1,1,109699.629029
2,2,87011.919929
3,3,67205.376427
4,4,119390.152002
...,...,...
31257,31257,104803.267809
31258,31258,99541.652636
31259,31259,175785.420722
31260,31260,251811.923734


In [72]:
df = pd.read_csv('test.csv')
id = df['id']
result = pd.concat([id, ensemble_result],axis=1)
result.to_csv('result.csv',index=False)

In [73]:
result

,id,0
0,31471,137317.640924
1,31472,109699.629029
2,31473,87011.919929
3,31474,67205.376427
4,31475,119390.152002
...,...,...
31257,62728,104803.267809
31258,62729,99541.652636
31259,62730,175785.420722
31260,62731,251811.923734


lightGBMにDARTとGOSSの予測値を特徴量として加えるアンサンブル。

In [13]:
DART_test

,id,0
0,31471,134385.720108
1,31472,121563.554862
2,31473,77357.960892
3,31474,60578.156482
4,31475,106603.223207
...,...,...
31257,62728,110081.947302
31258,62729,102139.722605
31259,62730,175842.715235
31260,62731,277540.405321


In [39]:
DART_pred = pd.read_csv('en_DART_cs_nLDKS_bld_result.csv')
DART_pred.columns = ['DARTの予測値']
DART_test = pd.read_csv('DART_cs_nLDKS_bld_result.csv')
DART_test = DART_test['0']
DART_test.columns = ['DARTの予測値']

GOSS_pred = pd.read_csv('en_GOSS_cs_nLDKS_bld_result.csv')
GOSS_pred.columns = ['GOSSの予測値']
GOSS_test = pd.read_csv('GOSS_cs_nLDKS_bld_result.csv')
GOSS_test = GOSS_test['0']
GOSS_test.columns = ['GOSSの予測値']

RF_pred = pd.read_csv('en_RF_cs2_result.csv')
RF_pred.columns = ['RFの予測値']
RF_test = pd.read_csv('RF_cs2_result.csv')
RF_test = RF_test['0']
RF_test.columns = ['RFの予測値']

MLR_pred = pd.read_csv('en_MLR_result.csv')
MLR_pred.columns = ['MLRの予測値']
MLR_test = pd.read_csv('MLR_result.csv')
MLR_test = MLR_test['0']
MLR_test.columns = ['MLRの予測値']

LGBM_pred = pd.read_csv('en_LGBM_result.csv')
LGBM_pred.columns = ['LGBMの予測値']
LGBM_test = pd.read_csv('LGBM_result.csv')
LGBM_test = LGBM_test['0']
LGBM_test.columns = ['LGBMの予測値']


In [59]:
X_train = pd.concat([house_age, Floor_scores2, DART_pred,GOSS_pred,RF_pred,MLR_pred,LGBM_pred], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([test_house_age,  test_Floor_scores2,  DART_test,GOSS_test,RF_pred,MLR_test,LGBM_test], axis=1)

category_lists = ['定期借家']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'learning_rate':0.05,
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    #categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
display(importance)


c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002603 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3068
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 7
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 44653	valid_1's rmse: 49122.2
[20]	training's rmse: 27651.3	valid_1's rmse: 35703.8
[30]	training's rmse: 17975.2	valid_1's rmse: 29070.9
[40]	training's rmse: 12820.9	valid_1's rmse: 26053.4
[50]	training's rmse: 10270.9	valid_1's rmse: 24728.8
[60]	training's rmse: 9133.63	valid_1's rmse: 24087.5
[70]	training's rmse: 8627.1	valid_1's rmse: 23766.1
[80]	training's rmse: 8371.29	valid_1's rmse: 23595.1
[90]	training's rmse: 8193.15	valid_1's rmse: 23428.1
[100]	training's rmse: 8070.55	valid_1's rmse: 23294.4
[110]	training's rmse: 7994.96	valid_1's rmse: 23186

,importance
築年数,3.321810e+12
全体の階数,3.576043e+12
DARTの予測値,7.843824e+14
GOSSの予測値,8.310645e+13
RFの予測値,7.602080e+12
MLRの予測値,2.771214e+12
LGBMの予測値,3.327038e+14


DARTとGOSSとランダムフォレストと重回帰分析の予測結果、および前にlightGBMから得た最良の結果だけをlightGBMの特徴量にしてみる。

In [58]:
X_train = pd.concat([DART_pred,GOSS_pred,RF_pred,MLR_pred,LGBM_pred], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([DART_test,GOSS_test,RF_pred,MLR_test,LGBM_test], axis=1)

category_lists = ['定期借家', 'L', 'D','K']
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective':'regression',
    'metrics':'rmse',
    'reg_lambda': 4.430375245218262e-06,
    'max_bin': 502,
    'num_leaves': 97
}

model = lgb.train(
                    params,
                    lgb_train, 
                    valid_sets=[lgb_train, lgb_eval], 
                    verbose_eval=10, 
                    num_boost_round=3000, 
                    early_stopping_rounds=10,
                    #categorical_feature = category_lists
                    )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(), index=X_train.columns, columns=['importance'])
display(importance)


c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000847 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2510
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 5
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 26973.1	valid_1's rmse: 35216.9
[20]	training's rmse: 12460	valid_1's rmse: 25867.1
[30]	training's rmse: 9197.61	valid_1's rmse: 24119.1
[40]	training's rmse: 8653.95	valid_1's rmse: 23702.8
[50]	training's rmse: 8546.62	valid_1's rmse: 23579.4
[60]	training's rmse: 8496.38	valid_1's rmse: 23541.3
[70]	training's rmse: 8463.76	valid_1's rmse: 23547.6
Early stopping, best iteration is:
[60]	training's rmse: 8496.38	valid_1's rmse: 23541.3


,importance
DARTの予測値,1686
GOSSの予測値,1095
RFの予測値,995
MLRの予測値,549
LGBMの予測値,1435


あまり上がらない・・・念のためにチューニングをしておく。

In [42]:
X_train = pd.concat([DART_pred,GOSS_pred,RF_pred,MLR_pred,LGBM_pred], axis=1)
y_train = rent

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

X_test = pd.concat([DART_test,GOSS_test,RF_pred,MLR_test,LGBM_test], axis=1)

def objective(trial):
    #category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', '建物構造']
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

    params = {
        'objective':'regression',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
        'max_bin': trial.suggest_int('max_bin', 255, 600),
        'num_leaves': trial.suggest_int('num_leaves', 50, 128),
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=1000, 
                        early_stopping_rounds=10,
                        #categorical_feature = category_lists
                        )

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)

    loss = mean_squared_error(y_valid, y_pred_valid, squared=False)
    return loss

In [43]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)
study.best_params

[I 2022-09-09 17:13:44,768] A new study created in memory with name: no-name-4365e651-165d-4c29-9b6a-2d19d3e74e8e
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000886 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2510
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 5
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 44653.5	valid_1's rmse: 49124.6
[20]	training's rmse: 27652.5	valid_1's rmse: 35711.5
[30]	training's rmse: 17977.4	valid_1's rmse: 29094.1
[40]	training's rmse: 12828.2	valid_1's rmse: 26074.1
[50]	training's rmse: 10363	valid_1's rmse: 24785.7
[60]	training's rmse: 9302.73	valid_1's rmse: 24192.3
[70]	training's rmse: 8858.56	valid_1's rmse: 23886.9
[80]	training's rmse: 8680.45	valid_1's rmse: 23726.9
[90]	training's rmse: 8605.34	valid_1's rmse: 23654.3
[100]	training's rmse: 8557.52	valid_1's rmse: 23599.4
[110]	training's rmse: 8524.38	valid_1's rmse: 2355

[I 2022-09-09 17:13:45,496] Trial 0 finished with value: 23549.216504023072 and parameters: {'reg_lambda': 4.430375245218262e-06, 'max_bin': 502, 'num_leaves': 97}. Best is trial 0 with value: 23549.216504023072.
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001041 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2005
[LightGBM] [Info] Number of data points in the train set: 22029, number of used features: 5
[LightGBM] [Info] Start training from score 118651.337373
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 44908.8	valid_1's rmse: 49397.1
[20]	training's rmse: 28175.4	valid_1's rmse: 36141.6
[30]	training's rmse: 18807.3	valid_1's rmse: 29740.1
[40]	training's rmse: 13965.8	valid_1's rmse: 26983.2
[50]	training's rmse: 11721.4	valid_1's rmse: 25832.6
[60]	training's rmse: 10785.3	valid_1's rmse: 25340
[70]	training's rmse: 10415.6	valid_1's rmse: 25115.6
[80]	training's rmse: 10262	valid_1's rmse: 24997.6
[90]	training's rmse: 10181.1	valid_1's rmse: 24910.7


[I 2022-09-09 17:13:46,174] Trial 1 finished with value: 24859.960466913097 and parameters: {'reg_lambda': 4.311710058685494e-06, 'max_bin': 401, 'num_leaves': 101}. Best is trial 0 with value: 23549.216504023072.


...

c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\koshi\python\signate\mynabi\venv\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[10]	training's rmse: 44796.6	valid_1's rmse: 49220
[20]	training's rmse: 27921.8	valid_1's rmse: 35998.6
[30]	training's rmse: 18311.3	valid_1's rmse: 29632.9
[40]	training's rmse: 12990.6	valid_1's rmse: 26691.7
[50]	training's rmse: 10276	valid_1's rmse: 25294.8
[60]	training's rmse: 9044.01	valid_1's rmse: 24631.9
[70]	training's rmse: 8515.97	valid_1's rmse: 24293.8
[80]	training's rmse: 8302.7	valid_1's rmse: 24123.4
[90]	training's rmse: 8216.26	valid_1's rmse: 24044.8
[100]	training's rmse: 8170.84	valid_1's rmse: 24012.9


[I 2022-09-09 17:14:24,413] Trial 39 finished with value: 23977.03706847359 and parameters: {'reg_lambda': 5.543941231681784e-06, 'max_bin': 560, 'num_leaves': 104}. Best is trial 37 with value: 23460.449289408472.


[110]	training's rmse: 8139.6	valid_1's rmse: 23982.2
[120]	training's rmse: 8119.27	valid_1's rmse: 23993.1
Early stopping, best iteration is:
[111]	training's rmse: 8137.3	valid_1's rmse: 23977


{'reg_lambda': 3.461194599494402e-05, 'max_bin': 497, 'num_leaves': 73}